In [0]:
from pyspark.ml.regression import LinearRegression
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("lrex").getOrCreate()

# Load training data
df = spark.read.format("csv").load("dbfs:/FileStore/shared_uploads/necmettinceylan@hotmail.com/cruise_ship_info.csv",
                                  header= True,
                                  inferSchema=True)
df.columns

Out[1]: ['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [0]:
df.printSchema()

root
-- Ship_name: string (nullable = true)
-- Cruise_line: string (nullable = true)
-- Age: integer (nullable = true)
-- Tonnage: double (nullable = true)
-- passengers: double (nullable = true)
-- length: double (nullable = true)
-- cabins: double (nullable = true)
-- passenger_density: double (nullable = true)
-- crew: double (nullable = true)

In [0]:
df.head(4)

Out[3]: [Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55),
 Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55),
 Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7),
 Row(Ship_name='Conquest', Cruise_line='Carnival', Age=11, Tonnage=110.0, passengers=29.74, length=9.53, cabins=14.88, passenger_density=36.99, crew=19.1)]

In [0]:
from pyspark.ml.feature import StringIndexer

In [0]:
indexer = StringIndexer( inputCol="Cruise_line", outputCol="cruise_cat" )
indexed = indexer.fit( df ).transform(df)
indexed.head(1)

Out[8]: [Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0)]

In [0]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler


In [0]:
indexed.columns

Out[11]: ['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'cruise_cat']

In [0]:
assembler = VectorAssembler(inputCols=['Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'cruise_cat'], outputCol="features")

output = assembler.transform(indexed)
output.columns

Out[13]: ['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'cruise_cat',
 'features']

In [0]:
output.select("features","crew").show()

+--------------------+----+
 features|crew|
+--------------------+----+
[6.0,30.276999999...|3.55|
[6.0,30.276999999...|3.55|
[26.0,47.262,14.8...| 6.7|
[11.0,110.0,29.74...|19.1|
[17.0,101.353,26....|10.0|
[22.0,70.367,20.5...| 9.2|
[15.0,70.367,20.5...| 9.2|
[23.0,70.367,20.5...| 9.2|
[19.0,70.367,20.5...| 9.2|
[6.0,110.23899999...|11.5|
[10.0,110.0,29.74...|11.6|
[28.0,46.052,14.5...| 6.6|
[18.0,70.367,20.5...| 9.2|
[17.0,70.367,20.5...| 9.2|
[11.0,86.0,21.24,...| 9.3|
[8.0,110.0,29.74,...|11.6|
[9.0,88.5,21.24,9...|10.3|
[15.0,70.367,20.5...| 9.2|
[12.0,88.5,21.24,...| 9.3|
[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows

In [0]:
final_data = output.select( ["features","crew" ])

In [0]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [0]:
train_data.describe().show()

+-------+------------------+
summary| crew|
+-------+------------------+
 count| 113|
 mean| 7.716814159292044|
 stddev|3.6426146578401655|
 min| 0.59|
 max| 21.0|
+-------+------------------+

In [0]:
from pyspark.ml.regression import LinearRegression

In [0]:
ship_lr = LinearRegression( labelCol="crew" )

In [0]:
trained_ship_model = ship_lr.fit(train_data)

In [0]:
test_ev = trained_ship_model.evaluate(test_data)
test_ev.rootMeanSquaredError

Out[24]: 0.6564950663970971

In [0]:
from pyspark.sql.functions import corr

In [0]:
df.select( corr("crew","passengers") ).show()

+----------------------+
corr(crew, passengers)|
+----------------------+
 0.9152341306065384|
+----------------------+

In [0]:
df.select( corr("crew","cabins") ).show()

+------------------+
corr(crew, cabins)|
+------------------+
0.9508226063578497|
+------------------+